In [1]:
import warnings
from typing import Any

# Standard imports
import numpy as np
import pandas as pd
import polars as pl

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)


def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [3]:
go_up_from_current_directory(go_up=1)

/Users/mac/Desktop/Projects/End-to-end-Sale-Forecasting


In [ ]:
from src.utilities.data_gen import RealisticSalesDataGenerator

gen_data = RealisticSalesDataGenerator(start_date="2025-08-29", end_date="2025-09-04", seed=123)
output_dict: dict[str, Any] = gen_data.generate_sales_data(output_dir="../data/sales_data")
output_dict

{'sales': ['../data/sales_data/sales/year=2025/month=08/day=30/sales_2025-08-30.parquet',
  '../data/sales_data/sales/year=2025/month=08/day=31/sales_2025-08-31.parquet',
  '../data/sales_data/sales/year=2025/month=09/day=01/sales_2025-09-01.parquet',
  '../data/sales_data/sales/year=2025/month=09/day=02/sales_2025-09-02.parquet',
  '../data/sales_data/sales/year=2025/month=09/day=03/sales_2025-09-03.parquet'],
 'inventory': ['../data/sales_data/inventory/year=2025/week=35/inventory_2025-08-31.parquet'],
 'customer_traffic': ['../data/sales_data/customer_traffic/year=2025/month=08/day=29/traffic_2025-08-29.parquet',
  '../data/sales_data/customer_traffic/year=2025/month=08/day=30/traffic_2025-08-30.parquet',
  '../data/sales_data/customer_traffic/year=2025/month=08/day=31/traffic_2025-08-31.parquet',
  '../data/sales_data/customer_traffic/year=2025/month=09/day=01/traffic_2025-09-01.parquet',
  '../data/sales_data/customer_traffic/year=2025/month=09/day=02/traffic_2025-09-02.parquet',


In [14]:
total_files = sum(len(paths) for paths in output_dict.values())
total_files

15

In [15]:
temp_pandas: pd.DataFrame = pd.read_parquet(output_dict["sales"][:3])
temp_pandas

,date,store_id,product_id,category,quantity_sold,unit_price,discount_percent,revenue,cost,profit,year,month,day
0,2025-08-30,store_001,HOME_005,Home,1,39,0.00,39.00,23.40,15.60,2025,8,30
1,2025-08-30,store_001,SPRT_005,Sports,1,89,0.00,89.00,57.85,31.15,2025,8,30
2,2025-08-30,store_002,CLTH_001,Clothing,1,29,0.00,29.00,14.50,14.50,2025,8,30
3,2025-08-31,store_001,CLTH_001,Clothing,1,29,0.00,29.00,14.50,14.50,2025,8,31
4,2025-08-31,store_001,CLTH_004,Clothing,1,89,0.00,89.00,46.28,42.72,2025,8,31
5,2025-08-31,store_001,HOME_002,Home,1,49,0.00,49.00,31.85,17.15,2025,8,31
6,2025-08-31,store_001,HOME_005,Home,1,39,0.00,39.00,23.40,15.60,2025,8,31
7,2025-08-31,store_001,SPRT_001,Sports,1,29,0.00,29.00,13.05,15.95,2025,8,31
8,2025-08-31,store_001,SPRT_002,Sports,1,49,0.00,49.00,26.95,22.05,2025,8,31
9,2025-08-31,store_009,CLTH_001,Clothing,1,29,0.00,29.00,14.50,14.50,2025,8,31


In [56]:
temp_df: pl.DataFrame = pl.read_parquet(output_dict["sales"][:3])
temp_df = temp_df.with_columns(pl.col("date").cast(pl.Date).alias("date"))
temp_df

date,store_id,product_id,category,quantity_sold,unit_price,discount_percent,revenue,cost,profit
date,str,str,str,i64,i64,f64,f64,f64,f64
2025-08-30,"""store_001""","""HOME_005""","""Home""",1,39,0.0,39.0,23.4,15.6
2025-08-30,"""store_001""","""SPRT_005""","""Sports""",1,89,0.0,89.0,57.85,31.15
2025-08-30,"""store_002""","""CLTH_001""","""Clothing""",1,29,0.0,29.0,14.5,14.5
2025-08-31,"""store_001""","""CLTH_001""","""Clothing""",1,29,0.0,29.0,14.5,14.5
2025-08-31,"""store_001""","""CLTH_004""","""Clothing""",1,89,0.0,89.0,46.28,42.72
2025-08-31,"""store_001""","""HOME_002""","""Home""",1,49,0.0,49.0,31.85,17.15
2025-08-31,"""store_001""","""HOME_005""","""Home""",1,39,0.0,39.0,23.4,15.6
2025-08-31,"""store_001""","""SPRT_001""","""Sports""",1,29,0.0,29.0,13.05,15.95
2025-08-31,"""store_001""","""SPRT_002""","""Sports""",1,49,0.0,49.0,26.95,22.05


In [ ]:
import holidays

us_holidays = holidays.UnitedStates()

temp_df = temp_df.with_columns(
    pl.col("date").dt.day().alias("day"),
    pl.col("date").dt.month().alias("month"),
    pl.col("date").dt.year().alias("year"),
    (pl.col("date").dt.weekday() - 1).alias("day_of_week"),
    pl.col("date").dt.quarter().alias("quarter"),
    pl.col("date").dt.week().alias("week_of_year"),
    ((pl.col("date").dt.weekday() - 1) >= 5).alias("is_weekend").cast(pl.Int8),
    pl.col("date").map_elements(lambda x: x in us_holidays).alias("is_holiday").cast(pl.Int8),
)
temp_df

date,store_id,product_id,category,quantity_sold,unit_price,discount_percent,revenue,cost,profit,day,month,year,day_of_week,quarter,week_of_year,is_weekend,is_holiday
date,str,str,str,i64,i64,f64,f64,f64,f64,i8,i8,i32,i8,i8,i8,i8,i8
2025-08-30,"""store_001""","""HOME_005""","""Home""",1,39,0.0,39.0,23.4,15.6,30,8,2025,5,3,35,1,0
2025-08-30,"""store_001""","""SPRT_005""","""Sports""",1,89,0.0,89.0,57.85,31.15,30,8,2025,5,3,35,1,0
2025-08-30,"""store_002""","""CLTH_001""","""Clothing""",1,29,0.0,29.0,14.5,14.5,30,8,2025,5,3,35,1,0
2025-08-31,"""store_001""","""CLTH_001""","""Clothing""",1,29,0.0,29.0,14.5,14.5,31,8,2025,6,3,35,1,0
2025-08-31,"""store_001""","""CLTH_004""","""Clothing""",1,89,0.0,89.0,46.28,42.72,31,8,2025,6,3,35,1,0
2025-08-31,"""store_001""","""HOME_002""","""Home""",1,49,0.0,49.0,31.85,17.15,31,8,2025,6,3,35,1,0
2025-08-31,"""store_001""","""HOME_005""","""Home""",1,39,0.0,39.0,23.4,15.6,31,8,2025,6,3,35,1,0
2025-08-31,"""store_001""","""SPRT_001""","""Sports""",1,29,0.0,29.0,13.05,15.95,31,8,2025,6,3,35,1,0
2025-08-31,"""store_001""","""SPRT_002""","""Sports""",1,49,0.0,49.0,26.95,22.05,31,8,2025,6,3,35,1,0


In [19]:
temp_pandas

,date,store_id,product_id,category,quantity_sold,unit_price,discount_percent,revenue,cost,profit,year,month,day
0,2025-08-30,store_001,HOME_005,Home,1,39,0.00,39.00,23.40,15.60,2025,8,30
1,2025-08-30,store_001,SPRT_005,Sports,1,89,0.00,89.00,57.85,31.15,2025,8,30
2,2025-08-30,store_002,CLTH_001,Clothing,1,29,0.00,29.00,14.50,14.50,2025,8,30
3,2025-08-31,store_001,CLTH_001,Clothing,1,29,0.00,29.00,14.50,14.50,2025,8,31
4,2025-08-31,store_001,CLTH_004,Clothing,1,89,0.00,89.00,46.28,42.72,2025,8,31
5,2025-08-31,store_001,HOME_002,Home,1,49,0.00,49.00,31.85,17.15,2025,8,31
6,2025-08-31,store_001,HOME_005,Home,1,39,0.00,39.00,23.40,15.60,2025,8,31
7,2025-08-31,store_001,SPRT_001,Sports,1,29,0.00,29.00,13.05,15.95,2025,8,31
8,2025-08-31,store_001,SPRT_002,Sports,1,49,0.00,49.00,26.95,22.05,2025,8,31
9,2025-08-31,store_009,CLTH_001,Clothing,1,29,0.00,29.00,14.50,14.50,2025,8,31


In [20]:
from src.config import app_config

In [ ]:
import polars.selectors as cs

n_cols: list[str] = temp_df.select(cs.numeric()).columns
for col in n_cols:
    if temp_df[col].is_null().any():
        console.print(f"Column '{col}' has missing values.", style="warning")
    else:
        console.print(f"Column '{col}' has no missing values.", style="success")


False

In [111]:
import polars.selectors as cs
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder


class FeatureEngineer:
    def __init__(self) -> None:
        self.feature_config = app_config.features
        self.validation_config = app_config.validation

    def create_date_features(
        self, df: pl.DataFrame, date_col: str = "date"
    ) -> pl.DataFrame:
        """Create date features from a date column.
        Note
        ----
        Polars by default uses a non-zero-based index for date components.
        """
        date_features: list[str] = self.feature_config.date_features
        df = df.clone()

        # Convert to datetime
        try:
            df = df.with_columns(pl.col("date").cast(pl.Date).alias("date"))
        except Exception as e:
            print(f"Error converting {date_col} to datetime: {e}")
            pass

        if "date" in date_features:
            return df.with_columns(
                pl.col("date").dt.day().alias("day"),
                pl.col("date").dt.month().alias("month"),
                pl.col("date").dt.year().alias("year"),
                (pl.col("date").dt.weekday() - 1).alias("day_of_week"),
                pl.col("date").dt.weekday().alias("day_of_week"),
                pl.col("date").dt.quarter().alias("quarter"),
                pl.col("date").dt.week().alias("week_of_year"),
                ((pl.col("date").dt.weekday() - 1) >= 5)
                .alias("is_weekend")
                .cast(pl.Int8),
                pl.col("date")
                .map_elements(lambda x: x in us_holidays)
                .alias("is_holiday")
                .cast(pl.Int8),
            )
        return df

    def create_lag_features(
        self, df: pl.DataFrame, target_col: str, group_cols: list[str] | None = None
    ) -> pl.DataFrame:
        df = df.clone()
        lag_values = self.feature_config.lag_features

        if group_cols:
            for lag in lag_values:
                df = df.with_columns(
                    pl.col(target_col)
                    .shift(lag)
                    .over(group_cols)
                    .alias(f"{target_col}_lag_{lag}")
                )
        else:
            for lag in lag_values:
                df = df.with_columns(
                    pl.col(target_col).shift(lag).alias(f"{target_col}_lag_{lag}")
                )

        print(f"Created {len(lag_values)} lag features")
        return df

    def create_rolling_features(
        self, df: pl.DataFrame, target_col: str, group_cols: list[str] | None = None
    ) -> pl.DataFrame:
        df = df.clone()
        windows = self.feature_config.rolling_features["windows"]
        functions = self.feature_config.rolling_features["functions"]

        if group_cols:
            for window in windows:
                for func in functions:
                    col_name: str = f"{target_col}_rolling_{window}_{func}"
                    if func == "mean":
                        df = df.with_columns(
                            pl.col(target_col)
                            .rolling_mean(window, min_samples=1)
                            .over(group_cols)
                            .alias(col_name)
                        )
                    if func == "std":
                        df = df.with_columns(
                            pl.col(target_col)
                            .rolling_std(window, min_samples=1)
                            .over(group_cols)
                            .alias(col_name)
                        )
                    if func == "std":
                        df = df.with_columns(
                            pl.col(target_col)
                            .rolling_std(window, min_samples=1)
                            .over(group_cols)
                            .alias(col_name)
                        )
                    if func == "min":
                        df = df.with_columns(
                            pl.col(target_col)
                            .rolling_min(window, min_samples=1)
                            .over(group_cols)
                            .alias(col_name)
                        )
                    if func == "max":
                        df = df.with_columns(
                            pl.col(target_col)
                            .rolling_max(window, min_samples=1)
                            .over(group_cols)
                            .alias(col_name)
                        )
        else:
            for window in windows:
                for func in functions:
                    col_name = f"{target_col}_rolling_{window}_{func}"
                    if func == "mean":
                        df = df.with_columns(
                            pl.col(target_col)
                            .rolling_mean(window, min_samples=1)
                            .alias(col_name)
                        )
                    if func == "std":
                        df = df.with_columns(
                            pl.col(target_col)
                            .rolling_std(window, min_samples=1)
                            .alias(col_name)
                        )
                    if func == "std":
                        df = df.with_columns(
                            pl.col(target_col)
                            .rolling_std(window, min_samples=1)
                            .alias(col_name)
                        )
                    if func == "min":
                        df = df.with_columns(
                            pl.col(target_col)
                            .rolling_min(window, min_samples=1)
                            .alias(col_name)
                        )
                    if func == "max":
                        df = df.with_columns(
                            pl.col(target_col)
                            .rolling_max(window, min_samples=1)
                            .alias(col_name)
                        )
        return df

    def create_interaction_features(
        self, df: pl.DataFrame, categorical_cols: list[str]
    ) -> pl.DataFrame:
        df = df.clone()

        for i, col1 in enumerate(categorical_cols):
            for col2 in categorical_cols[i + 1 :]:
                df = df.with_columns(
                    (
                        pl.col(col1).cast(pl.Utf8) + "_" + pl.col(col2).cast(pl.Utf8)
                    ).alias(f"{col1}_{col2}_interaction")
                )

        return df

    def create_cyclical_features(
        self, df: pl.DataFrame, date_col: str = "date"
    ) -> pl.DataFrame:
        df = df.clone()

        return df.with_columns(
            # month (convert 1-12 to 0-11 for proper cyclical encoding)
            pl.col(date_col)
            .dt.month()
            .map_elements(lambda x: np.sin(2 * np.pi * (x - 1) / 12))
            .alias("month_sin"),
            pl.col(date_col)
            .dt.month()
            .map_elements(lambda x: np.cos(2 * np.pi * (x - 1) / 12))
            .alias("month_cos"),
            # day (Retain original values; 1-31)
            pl.col(date_col)
            .dt.day()
            .map_elements(lambda x: np.sin(2 * np.pi * x / 31))
            .alias("day_sin"),
            pl.col(date_col)
            .dt.day()
            .map_elements(lambda x: np.cos(2 * np.pi * x / 31))
            .alias("day_cos"),
            # day of week (convert 1-7 to 0-6 for proper cyclical encoding)
            pl.col(date_col)
            .dt.weekday()
            .map_elements(lambda x: np.sin(2 * np.pi * (x - 1) / 7))
            .alias("day_of_week_sin"),
            pl.col(date_col)
            .dt.weekday()
            .map_elements(lambda x: np.cos(2 * np.pi * (x - 1) / 7))
            .alias("day_of_week_cos"),
        )

    def handle_missing_values(self, df: pl.DataFrame) -> pl.DataFrame:
        numeric_columns: list[str] = df.select(cs.numeric()).columns

        for col in numeric_columns:
            if df[col].is_null().any():
                if "lag" in col or "rolling" in col:
                    # Forward fill then backward fill
                    df = df.with_columns(
                        pl.col(col)
                        .fill_null(strategy="forward")
                        .fill_null(strategy="backward")
                    )
                else:
                    # For other columns, just use average
                    df = df.with_columns(pl.col(col).fill_null(strategy="mean"))
        return df

    def create_all_features(
        self,
        df: pl.DataFrame,
        target_col: str = "sales",
        date_col: str = "date",
        group_cols: list[str] | None = None,
        categorical_cols: list[str] | None = None,
    ) -> pl.DataFrame:
        print("Starting feature engineering pipeline")

        if group_cols:
            df = df.sort(by=group_cols + [date_col])
        else:
            df = df.sort(by=[date_col])

        # Create date features
        df = self.create_date_features(df, date_col=date_col)

        # Create lag features
        df = self.create_lag_features(df, target_col=target_col, group_cols=group_cols)

        # Create rolling features
        df = self.create_rolling_features(
            df, target_col=target_col, group_cols=group_cols
        )

        # Create cyclical features
        df = self.create_cyclical_features(df, date_col=date_col)

        # Create interaction features
        if categorical_cols:
            df = self.create_interaction_features(df, categorical_cols=categorical_cols)

        # Handle missing values
        df = self.handle_missing_values(df)

        print(
            f"Feature engineering pipeline completed. {len(df.columns)!r} total features."
        )

        return df

    def select_features(
        self, df: pl.DataFrame, target_col: str, importance_threshold: float = 0.001
    ) -> list[str]:
        X = df.drop([target_col])
        y = df[target_col]
        label_encoders: dict[str, LabelEncoder] = {}

        # Encode categorical variables
        for col in X.select(cs.categorical()).columns:
            le = LabelEncoder()
            values = le.fit_transform(X[col])
            df = df.with_columns(pl.Series(col, values=values, dtype=pl.Int8))
            label_encoders[col] = le

        # Train the model
        rf = RandomForestRegressor(n_estimators=50, random_state=42)
        rf.fit(X, y)

        # Get feature importances
        importances = pl.DataFrame(
            {"feature": X.columns, "importance": rf.feature_importances_}
        ).sort("importance", descending=True)
        # Select features based on importance
        return importances.filter(pl.col("importance") > importance_threshold)[
            "feature"
        ].to_list()

In [104]:
df: pl.DataFrame = pl.DataFrame(
    data={
        "id": [1, 2, 3, 4],
        "name": ["Alice", "Bob", "Charlie", "Bob"],
        "role": ["Engineer", "Manager", "Engineer", "Manager"],
        "experience": [5, 2, 3, 3],
        "age": [30, 40, 35, 34],
        "target": [1, 0, 1, 0],
    }
)

df

id,name,role,experience,age,target
i64,str,str,i64,i64,i64
1,"""Alice""","""Engineer""",5,30,1
2,"""Bob""","""Manager""",2,40,0
3,"""Charlie""","""Engineer""",3,35,1
4,"""Bob""","""Manager""",3,34,0


In [110]:
X = df.drop(["target", "id"])
y = df["target"]

label_encoders: dict[str, LabelEncoder] = {}


for col in ["name", "role"]:
    le = LabelEncoder()
    vals = le.fit_transform(X[col])

    X = X.with_columns(pl.Series(col, values=vals, dtype=pl.Int8))
    label_encoders[col] = le

# Train the model
rf = RandomForestRegressor(n_estimators=50, random_state=42)
rf.fit(X, y)

# Get feature importances
importances = pl.DataFrame(
    {"feature": X.columns, "importance": rf.feature_importances_}
).sort("importance", descending=True)


display(importances)

importances.filter(pl.col("importance") > 0.05)["feature"].to_list()

feature,importance
str,f64
"""role""",0.452381
"""name""",0.285714
"""age""",0.214286
"""experience""",0.047619


['role', 'name', 'age']

In [94]:
df

id,name,role
i64,i8,i8
1,0,0
2,1,1
3,2,0
4,1,1


In [114]:
data_gen = FeatureEngineer()
# Fix this!
# data_gen.create_all_features(temp_df, date_col="date", target_col="revenue")
data_gen.select_features(temp_df, target_col="revenue")

TypeError: float() argument must be a string or a real number, not 'datetime.date'

In [ ]:
from typing import List, Optional


class FeatureEngineerPandas:
    def __init__(self) -> None:
        self.feature_config = app_config.features
        self.validation_config = app_config.validation

    def create_date_features(self, df: pd.DataFrame, date_col: str = "date") -> pd.DataFrame:
        df = df.copy()

        df[date_col] = pd.to_datetime(df[date_col])

        date_features = self.feature_config.date_features

        if "year" in date_features:
            df["year"] = df[date_col].dt.year
        if "month" in date_features:
            df["month"] = df[date_col].dt.month
        if "day" in date_features:
            df["day"] = df[date_col].dt.day
        if "dayofweek" in date_features:
            df["dayofweek"] = df[date_col].dt.dayofweek
        if "quarter" in date_features:
            df["quarter"] = df[date_col].dt.quarter
        if "weekofyear" in date_features:
            df["weekofyear"] = df[date_col].dt.isocalendar().week
        if "is_weekend" in date_features:
            df["is_weekend"] = (df[date_col].dt.dayofweek >= 5).astype(int)
        if "is_holiday" in date_features:
            us_holidays = holidays.US()
            df["is_holiday"] = df[date_col].apply(lambda x: x in us_holidays).astype(int)

        print(f"Created {len(date_features)} date features")
        return df

    def create_lag_features(self, df: pd.DataFrame, target_col: str, group_cols: Optional[List[str]] = None) -> pd.DataFrame:
        df = df.copy()
        lag_values = self.feature_config.lag_features

        if group_cols:
            for lag in lag_values:
                df[f"{target_col}_lag_{lag}"] = df.groupby(group_cols)[target_col].shift(lag)
        else:
            for lag in lag_values:
                df[f"{target_col}_lag_{lag}"] = df[target_col].shift(lag)

        print(f"Created {len(lag_values)} lag features")
        return df

    def create_rolling_features(
        self, df: pd.DataFrame, target_col: str, group_cols: Optional[List[str]] = None
    ) -> pd.DataFrame:
        df = df.copy()
        windows = self.feature_config.rolling_features["windows"]
        functions = self.feature_config.rolling_features["functions"]

        if group_cols:
            for window in windows:
                for func in functions:
                    col_name = f"{target_col}_rolling_{window}_{func}"
                    df[col_name] = df.groupby(group_cols)[target_col].transform(
                        lambda x: x.rolling(window, min_periods=1).agg(func)
                    )
        else:
            for window in windows:
                for func in functions:
                    col_name = f"{target_col}_rolling_{window}_{func}"
                    df[col_name] = df[target_col].rolling(window, min_periods=1).agg(func)

        print(f"Created {len(windows) * len(functions)} rolling features")
        return df

    def create_interaction_features(self, df: pd.DataFrame, categorical_cols: List[str]) -> pd.DataFrame:
        df = df.copy()

        for i, col1 in enumerate(categorical_cols):
            for col2 in categorical_cols[i + 1 :]:
                df[f"{col1}_{col2}_interaction"] = df[col1].astype(str) + "_" + df[col2].astype(str)

        return df

    def create_cyclical_features(self, df: pd.DataFrame, date_col: str = "date") -> pd.DataFrame:
        df = df.copy()

        df["month_sin"] = np.sin(2 * np.pi * df[date_col].dt.month / 12)
        df["month_cos"] = np.cos(2 * np.pi * df[date_col].dt.month / 12)

        df["day_sin"] = np.sin(2 * np.pi * df[date_col].dt.day / 31)
        df["day_cos"] = np.cos(2 * np.pi * df[date_col].dt.day / 31)

        df["dayofweek_sin"] = np.sin(2 * np.pi * df[date_col].dt.dayofweek / 7)
        df["dayofweek_cos"] = np.cos(2 * np.pi * df[date_col].dt.dayofweek / 7)

        print("Created cyclical features")
        return df

    def create_all_features(
        self,
        df: pd.DataFrame,
        target_col: str = "sales",
        date_col: str = "date",
        group_cols: Optional[List[str]] = None,
        categorical_cols: Optional[List[str]] = None,
    ) -> pd.DataFrame:
        print("Starting feature engineering pipeline")

        # Sort by date for proper lag and rolling calculations
        if group_cols:
            df = df.sort_values(group_cols + [date_col])
        else:
            df = df.sort_values(date_col)

        # Create date features
        df = self.create_date_features(df, date_col)

        # Create lag features
        df = self.create_lag_features(df, target_col, group_cols)

        # Create rolling features
        df = self.create_rolling_features(df, target_col, group_cols)

        # Create cyclical features
        df = self.create_cyclical_features(df, date_col)

        # Create interaction features if categorical columns provided
        if categorical_cols:
            df = self.create_interaction_features(df, categorical_cols)

        # Skip advanced features for now to reduce complexity
        # df = self.create_advanced_features(df, target_col, date_col, group_cols)

        # Handle missing values created by lag and rolling features
        df = self.handle_missing_values(df)

        print(f"Feature engineering complete. Total features: {len(df.columns)}")
        return df

    def handle_missing_values(self, df: pd.DataFrame) -> pd.DataFrame:
        # For lag and rolling features, forward fill or use mean
        numeric_columns = df.select_dtypes(include=[np.number]).columns

        for col in numeric_columns:
            if df[col].isnull().any():
                if "lag" in col or "rolling" in col:
                    # For time-based features, forward fill then backward fill
                    df[col] = df[col].ffill().bfill()
                else:
                    # For other features, use mean
                    df[col] = df[col].fillna(df[col].mean())

        return df

    def select_features(self, df: pd.DataFrame, target_col: str, importance_threshold: float = 0.001) -> List[str]:
        # Prepare data for feature selection
        X = df.drop(columns=[target_col])
        y = df[target_col]

        # Encode categorical variables
        label_encoders = {}
        for col in X.select_dtypes(include=["object"]).columns:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col].astype(str))
            label_encoders[col] = le

        # Train random forest for feature importance
        rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        rf.fit(X, y)

        # Get feature importances
        feature_importance = pd.DataFrame({"feature": X.columns, "importance": rf.feature_importances_}).sort_values(
            "importance", ascending=False
        )

        # Select features above threshold
        selected_features = feature_importance[feature_importance["importance"] >= importance_threshold]["feature"].tolist()

        print(f"Selected {len(selected_features)} features out of {len(X.columns)}")
        return selected_features

    def create_advanced_features(
        self,
        df: pd.DataFrame,
        target_col: str,
        date_col: str,
        group_cols: Optional[List[str]] = None,
    ) -> pd.DataFrame:
        """Create advanced features for better model performance"""
        df = df.copy()

        # Exponentially weighted moving averages (more weight on recent data)
        ewm_spans = [7, 14]  # Reduced spans to avoid overfitting
        for span in ewm_spans:
            if group_cols:
                df[f"{target_col}_ewm_{span}"] = df.groupby(group_cols)[target_col].transform(
                    lambda x: x.ewm(span=span, adjust=False).mean()
                )
            else:
                df[f"{target_col}_ewm_{span}"] = df[target_col].ewm(span=span, adjust=False).mean()

        # Trend features
        if group_cols:
            # Linear trend within groups
            df["trend"] = df.groupby(group_cols).cumcount()
            df["trend_squared"] = df["trend"] ** 2
        else:
            df["trend"] = np.arange(len(df))
            df["trend_squared"] = df["trend"] ** 2

        # Sales velocity (rate of change)
        df[f"{target_col}_velocity"] = df[target_col].diff()
        df[f"{target_col}_acceleration"] = df[f"{target_col}_velocity"].diff()

        # Ratio features
        for window in [7, 30]:
            rolling_mean = df[target_col].rolling(window, min_periods=1).mean()
            df[f"{target_col}_ratio_to_{window}d_avg"] = df[target_col] / (rolling_mean + 1)

        # Day of month features
        df["day_of_month"] = df[date_col].dt.day
        df["is_month_start"] = (df["day_of_month"] <= 5).astype(int)
        df["is_month_end"] = (df["day_of_month"] >= 25).astype(int)

        # Week of month
        df["week_of_month"] = (df["day_of_month"] - 1) // 7 + 1

        # Business quarter features
        df["quarter_progress"] = (df[date_col].dt.month - 1) % 3 + 1
        df["is_quarter_end"] = (df["quarter_progress"] == 3).astype(int)

        # Add carefully selected features that improve time series prediction
        if "has_promotion" in df.columns and "is_weekend" in df.columns:
            # Simple interaction between promotion and weekend
            df["promotion_weekend"] = df["has_promotion"] * df["is_weekend"]

        # Ratio features that capture relative performance
        for window in [7, 30]:
            rolling_mean = df[target_col].rolling(window, min_periods=1).mean()
            df[f"{target_col}_ratio_to_{window}d"] = df[target_col] / (rolling_mean + 1)

        # Days since month start (useful for monthly patterns)
        df["days_since_month_start"] = df["day_of_month"]

        print("Created advanced features")
        return df

    def create_target_encoding(
        self,
        df: pd.DataFrame,
        target_col: str,
        categorical_cols: List[str],
        smoothing: float = 1.0,
    ) -> pd.DataFrame:
        """Create target encoding for categorical variables with smoothing"""
        df = df.copy()

        for col in categorical_cols:
            # Calculate mean target for each category
            mean_target = df.groupby(col)[target_col].mean()
            global_mean = df[target_col].mean()

            # Calculate counts for smoothing
            counts = df[col].value_counts()

            # Apply smoothing to prevent overfitting
            smooth_mean = {}
            for cat in counts.index:
                n = counts[cat]
                smooth_mean[cat] = (n * mean_target[cat] + smoothing * global_mean) / (n + smoothing)

            # Create new feature
            df[f"{col}_target_encoded"] = df[col].map(smooth_mean)

            # Handle unknown categories
            df[f"{col}_target_encoded"].fillna(global_mean, inplace=True)

        print(f"Created target encoding for {len(categorical_cols)} categorical features")
        return df


pandas_data_gen = FeatureEngineerPandas()
pandas_data_gen.create_all_features(temp_pandas, date_col="date", target_col="revenue")
# pandas_data_gen.create_rolling_features(
#     temp_pandas, target_col="revenue", group_cols=["store_id"]
# )

Starting feature engineering pipeline
Created 8 date features
Created 7 lag features
Created 25 rolling features
Created cyclical features
Feature engineering complete. Total features: 54


,date,store_id,product_id,category,quantity_sold,unit_price,discount_percent,revenue,cost,profit,year,month,day,quarter,is_weekend,is_holiday,revenue_lag_1,revenue_lag_2,revenue_lag_3,revenue_lag_7,revenue_lag_14,revenue_lag_21,revenue_lag_30,revenue_rolling_3_mean,revenue_rolling_3_std,revenue_rolling_3_min,revenue_rolling_3_max,revenue_rolling_3_median,revenue_rolling_7_mean,revenue_rolling_7_std,revenue_rolling_7_min,revenue_rolling_7_max,revenue_rolling_7_median,revenue_rolling_14_mean,revenue_rolling_14_std,revenue_rolling_14_min,revenue_rolling_14_max,revenue_rolling_14_median,revenue_rolling_21_mean,revenue_rolling_21_std,revenue_rolling_21_min,revenue_rolling_21_max,revenue_rolling_21_median,revenue_rolling_30_mean,revenue_rolling_30_std,revenue_rolling_30_min,revenue_rolling_30_max,revenue_rolling_30_median,month_sin,month_cos,day_sin,day_cos,dayofweek_sin,dayofweek_cos
0,2025-08-30,store_001,HOME_005,Home,1,39,0.00,39.00,23.40,15.60,2025,8,30,3,1,0,39.0,39.0,39.0,39.0,NaN,NaN,NaN,39.000000,35.355339,39.0,39.00,39.0,39.000000,35.355339,39.0,39.00,39.0,39.000000,35.355339,39.0,39.0,39.0,39.000000,35.355339,39.0,39.0,39.0,39.000000,35.355339,39.0,39.0,39.0,-0.866025,-5.000000e-01,-2.012985e-01,0.97953,-0.974928,-0.222521
1,2025-08-30,store_001,SPRT_005,Sports,1,89,0.00,89.00,57.85,31.15,2025,8,30,3,1,0,39.0,39.0,39.0,39.0,NaN,NaN,NaN,64.000000,35.355339,39.0,89.00,64.0,64.000000,35.355339,39.0,89.00,64.0,64.000000,35.355339,39.0,89.0,64.0,64.000000,35.355339,39.0,89.0,64.0,64.000000,35.355339,39.0,89.0,64.0,-0.866025,-5.000000e-01,-2.012985e-01,0.97953,-0.974928,-0.222521
2,2025-08-30,store_002,CLTH_001,Clothing,1,29,0.00,29.00,14.50,14.50,2025,8,30,3,1,0,89.0,39.0,39.0,39.0,NaN,NaN,NaN,52.333333,32.145503,29.0,89.00,39.0,52.333333,32.145503,29.0,89.00,39.0,52.333333,32.145503,29.0,89.0,39.0,52.333333,32.145503,29.0,89.0,39.0,52.333333,32.145503,29.0,89.0,39.0,-0.866025,-5.000000e-01,-2.012985e-01,0.97953,-0.974928,-0.222521
3,2025-08-31,store_001,CLTH_001,Clothing,1,29,0.00,29.00,14.50,14.50,2025,8,31,3,1,0,29.0,89.0,39.0,39.0,NaN,NaN,NaN,49.000000,34.641016,29.0,89.00,29.0,46.500000,28.722813,29.0,89.00,34.0,46.500000,28.722813,29.0,89.0,34.0,46.500000,28.722813,29.0,89.0,34.0,46.500000,28.722813,29.0,89.0,34.0,-0.866025,-5.000000e-01,-2.449294e-16,1.00000,-0.781831,0.623490
4,2025-08-31,store_001,CLTH_004,Clothing,1,89,0.00,89.00,46.28,42.72,2025,8,31,3,1,0,29.0,29.0,89.0,39.0,NaN,NaN,NaN,49.000000,34.641016,29.0,89.00,29.0,55.000000,31.304952,29.0,89.00,39.0,55.000000,31.304952,29.0,89.0,39.0,55.000000,31.304952,29.0,89.0,39.0,55.000000,31.304952,29.0,89.0,39.0,-0.866025,-5.000000e-01,-2.449294e-16,1.00000,-0.781831,0.623490
5,2025-08-31,store_001,HOME_002,Home,1,49,0.00,49.00,31.85,17.15,2025,8,31,3,1,0,89.0,29.0,29.0,39.0,NaN,NaN,NaN,55.666667,30.550505,29.0,89.00,49.0,54.000000,28.106939,29.0,89.00,44.0,54.000000,28.106939,29.0,89.0,44.0,54.000000,28.106939,29.0,89.0,44.0,54.000000,28.106939,29.0,89.0,44.0,-0.866025,-5.000000e-01,-2.449294e-16,1.00000,-0.781831,0.623490
6,2025-08-31,store_001,HOME_005,Home,1,39,0.00,39.00,23.40,15.60,2025,8,31,3,1,0,49.0,89.0,29.0,39.0,NaN,NaN,NaN,59.000000,26.457513,39.0,89.00,49.0,51.857143,26.276914,29.0,89.00,39.0,51.857143,26.276914,29.0,89.0,39.0,51.857143,26.276914,29.0,89.0,39.0,51.857143,26.276914,29.0,89.0,39.0,-0.866025,-5.000000e-01,-2.449294e-16,1.00000,-0.781831,0.623490
7,2025-08-31,store_001,SPRT_001,Sports,1,29,0.00,29.00,13.05,15.95,2025,8,31,3,1,0,39.0,49.0,89.0,39.0,NaN,NaN,NaN,39.000000,10.000000,29.0,49.00,39.0,50.428571,27.342623,29.0,89.00,39.0,49.000000,25.634798,29.0,89.0,39.0,49.000000,25.634798,29.0,89.0,39.0,49.000000,25.634798,29.0,89.0,39.0,-0.866025,-5.000000e-01,-2.449294e-16,1.00000,-0.781831,0.623490
8,2025-08-31,store_001,SPRT_002,Sports,1,49,0.00,49.00,26.95,22.05,2025,8,31,3,1,0,29.0,39.0,49.0,89.0,NaN,NaN,NaN,39.000000,10.000000,29.0,49.00,39.0,44.714286,21.491970,29.0,89.00,39.0,49.000000,23.979158,29.0,89.0,39.0,49.000000,23.979158,29.0,89

In [ ]:
def create_cyclical_features(df: pl.DataFrame, date_col: str = "date") -> pl.DataFrame:
    df = df.clone()

    return df.with_columns(
        # month (convert 1-12 to 0-11 for proper cyclical encoding)
        pl.col(date_col).dt.month().map_elements(lambda x: np.sin(2 * np.pi * (x - 1) / 12)).alias("month_sin"),
        pl.col(date_col).dt.month().map_elements(lambda x: np.cos(2 * np.pi * (x - 1) / 12)).alias("month_cos"),
        # day (Retain original values; 1-31)
        pl.col(date_col).dt.day().map_elements(lambda x: np.sin(2 * np.pi * x / 31)).alias("day_sin"),
        pl.col(date_col).dt.day().map_elements(lambda x: np.cos(2 * np.pi * x / 31)).alias("day_cos"),
        # day of week (convert 1-7 to 0-6 for proper cyclical encoding)
        pl.col(date_col).dt.weekday().map_elements(lambda x: np.sin(2 * np.pi * (x - 1) / 7)).alias("day_of_week_sin"),
        pl.col(date_col).dt.weekday().map_elements(lambda x: np.cos(2 * np.pi * (x - 1) / 7)).alias("day_of_week_cos"),
    )


create_cyclical_features(temp_df, date_col="date")

date,store_id,product_id,category,quantity_sold,unit_price,discount_percent,revenue,cost,profit,day,month,year,day_of_week,quarter,week_of_year,is_weekend,is_holiday,month_sin,month_cos,day_sin,day_cos,day_of_week_sin,day_of_week_cos
date,str,str,str,i64,i64,f64,f64,f64,f64,i8,i8,i32,i8,i8,i8,i8,i8,f64,f64,f64,f64,f64,f64
2025-08-30,"""store_001""","""HOME_005""","""Home""",1,39,0.0,39.0,23.4,15.6,30,8,2025,5,3,35,1,0,-0.5,-0.866025,-0.394356,0.97953,-0.974928,-0.222521
2025-08-30,"""store_001""","""SPRT_005""","""Sports""",1,89,0.0,89.0,57.85,31.15,30,8,2025,5,3,35,1,0,-0.5,-0.866025,-0.394356,0.97953,-0.974928,-0.222521
2025-08-30,"""store_002""","""CLTH_001""","""Clothing""",1,29,0.0,29.0,14.5,14.5,30,8,2025,5,3,35,1,0,-0.5,-0.866025,-0.394356,0.97953,-0.974928,-0.222521
2025-08-31,"""store_001""","""CLTH_001""","""Clothing""",1,29,0.0,29.0,14.5,14.5,31,8,2025,6,3,35,1,0,-0.5,-0.866025,-0.201299,1.0,-0.781831,0.62349
2025-08-31,"""store_001""","""CLTH_004""","""Clothing""",1,89,0.0,89.0,46.28,42.72,31,8,2025,6,3,35,1,0,-0.5,-0.866025,-0.201299,1.0,-0.781831,0.62349
2025-08-31,"""store_001""","""HOME_002""","""Home""",1,49,0.0,49.0,31.85,17.15,31,8,2025,6,3,35,1,0,-0.5,-0.866025,-0.201299,1.0,-0.781831,0.62349
2025-08-31,"""store_001""","""HOME_005""","""Home""",1,39,0.0,39.0,23.4,15.6,31,8,2025,6,3,35,1,0,-0.5,-0.866025,-0.201299,1.0,-0.781831,0.62349
2025-08-31,"""store_001""","""SPRT_001""","""Sports""",1,29,0.0,29.0,13.05,15.95,31,8,2025,6,3,35,1,0,-0.5,-0.866025,-0.201299,1.0,-0.781831,0.62349
2025-08-31,"""store_001""","""SPRT_002""","""Sports""",1,49,0.0,49.0,26.95,22.05,31,8,2025,6,3,35,1,0,-0.5,-0.866025,-0.201299,1.0,-0.781831,0.62349


In [ ]:
# Let's examine the cyclical features to see what's wrong
cyclical_result = create_cyclical_features(temp_df, date_col="date")

# Check the cyclical features
cyclical_sample = cyclical_result.select(
    [
        "date",
        "day_of_week",
        "day_of_week_sin",
        "day_of_week_cos",
        "month_sin",
        "month_cos",
    ]
).unique()

print("Cyclical features sample:")
print(cyclical_sample)

print("\nLet's check the day_of_week values and corresponding sin/cos:")
day_check = (
    cyclical_result.select(["date", "day_of_week", "day_of_week_sin", "day_of_week_cos"]).unique().sort("day_of_week")
)
print(day_check)

print("\nIssue Analysis:")
print("day_of_week ranges from 1-7 in Polars (Monday=1, Sunday=7)")
print("But for cyclical encoding, we want values from 0 to 2π")
print("Current formula: sin(2π × day_of_week / 7)")
print("This means day 7 gives: sin(2π × 7 / 7) = sin(2π) = 0")
print("And day 1 gives: sin(2π × 1 / 7) = sin(2π/7)")
print("This creates a discontinuity between Sunday (7) and Monday (1)!")

In [ ]:
# The issue with day_of_week cyclical encoding:
print("PROBLEM IDENTIFIED:")
print("Polars weekday() returns 1-7 (Monday=1, Sunday=7)")
print("For proper cyclical encoding, we need 0-based indexing")
print()


def create_cyclical_features_corrected(df: pl.DataFrame, date_col: str = "date") -> pl.DataFrame:
    df = df.clone()

    return df.with_columns(
        # Month cyclical features (months 1-12, convert to 0-11)
        pl.col(date_col).dt.month().map_elements(lambda x: np.sin(2 * np.pi * (x - 1) / 12)).alias("month_sin"),
        pl.col(date_col).dt.month().map_elements(lambda x: np.cos(2 * np.pi * (x - 1) / 12)).alias("month_cos"),
        # Day cyclical features (days 1-31, convert to 0-30)
        pl.col(date_col).dt.day().map_elements(lambda x: np.sin(2 * np.pi * (x - 1) / 31)).alias("day_sin"),
        pl.col(date_col).dt.day().map_elements(lambda x: np.cos(2 * np.pi * (x - 1) / 31)).alias("day_cos"),
        # Day of week cyclical features (weekday 1-7, convert to 0-6)
        pl.col(date_col)
        .dt.weekday()
        .map_elements(lambda x: np.sin(2 * np.pi * (x - 1) / 7))  # CORRECTED: (x-1)
        .alias("day_of_week_sin"),
        pl.col(date_col)
        .dt.weekday()
        .map_elements(lambda x: np.cos(2 * np.pi * (x - 1) / 7))  # CORRECTED: (x-1)
        .alias("day_of_week_cos"),
    )


# Test the corrected function
print("Testing corrected cyclical features:")
corrected_result = create_cyclical_features_corrected(temp_df, date_col="date")

# Compare day_of_week encoding
comparison = (
    corrected_result.select(["date", "day_of_week", "day_of_week_sin", "day_of_week_cos"]).unique().sort("day_of_week")
)

print("\nCorrected day_of_week cyclical encoding:")
print(comparison)

print("\nKey differences:")
print("- Original: sin(2π × day_of_week / 7) where day_of_week ∈ [1,7]")
print("- Corrected: sin(2π × (day_of_week - 1) / 7) where (day_of_week - 1) ∈ [0,6]")
print("- This ensures Monday(1) → 0, Tuesday(2) → 1, ..., Sunday(7) → 6")
print("- Now Sunday(6) and Monday(0) are properly connected in the cycle!")

In [ ]:
# Test the corrected function
print("Testing the corrected create_cyclical_features function:")
corrected_cyclical = create_cyclical_features(temp_df, date_col="date")

# Show the cyclical features for each day of week
cyclical_summary = (
    corrected_cyclical.select(["date", "day_of_week", "day_of_week_sin", "day_of_week_cos"]).unique().sort("day_of_week")
)

print("\nCorrected cyclical features by day of week:")
print(cyclical_summary)

print("\nVerification:")
print("Monday (1) → sin=0.0, cos=1.0 (start of cycle)")
print("Sunday (7) → sin≈-0.78, cos≈0.62 (connects smoothly back to Monday)")
print("The cyclical encoding now properly represents the weekly cycle!")

In [ ]:
-2.4493e-16